# Data Preparation

This notebook contains code to collect and process source data to construct ML dataset for the project.

Content:
- Configuration cell (defines configuration for the following processing)
- Processing logic cell (contains functions with processing logic)
- Data processing workflow cells (data processing steps):
    1. Download a geographical shapefile with ZIP code service areas boundaries
    2. Download MODIS Land Surface Temperature data
    3. Download MODIS Normalized Difference Vegetation Index data
    4. Download satellite meteo data
    5. Download satellite Land Cover data
    6. Download satellite Fractional Impervious Surface data
    7. Download population density data
    8. Daily local weather data by city
    9. Process Land Surface Temperature data
    10. Process Normalized Difference Vegetation Index data
    11. Process satellite meteo data
    12. Process population density data
    13. Process local meteostat data
    14. process Land Cover data
    15. Process Normalized Difference Vegetation Index data
    16. Assemble final dataset

## Configuration

In [ ]:
import os
import zipfile
from io import BytesIO
import requests
import datetime
import geopandas as gpd
from tqdm import tqdm
from osgeo import gdal
import numpy as np
from rasterstats import zonal_stats
import glob
import pandas as pd
import xarray as xr
import rasterio
import warnings  

warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

NASA_EARTHDATA_TOKEN = """<your_nasa_earthdata_token>"""

PROJECT_DIR = os.getcwd()
DATA_DIR = os.path.join(PROJECT_DIR, 'datasets')
RAW_DATA_DIR = os.path.join(DATA_DIR, 'raw')
ZCTA_DATA_DIR = os.path.join(RAW_DATA_DIR, 'zcta')
ZCTA_SHAPEFILE= os.path.join(ZCTA_DATA_DIR, 'tl_2024_us_zcta520.shp')
LST_DATA_DIR = os.path.join(RAW_DATA_DIR, 'lst')
VI_DATA_DIR = os.path.join(RAW_DATA_DIR, 'vi')
DAYMET_DATA_DIR = os.path.join(RAW_DATA_DIR, 'daymet')
LC_DATA_DIR = os.path.join(RAW_DATA_DIR, 'lc')
FIS_DATA_DIR = os.path.join(RAW_DATA_DIR, 'fis')
CENSUS_DATA_DIR = os.path.join(RAW_DATA_DIR, 'census')
METEOSTAT_DATA_DIR = os.path.join(RAW_DATA_DIR, 'meteostat')

PROCESSED_DATA_DIR=os.path.join(DATA_DIR, 'processed')
PROCESSED_LST_DATA_DIR = os.path.join(PROCESSED_DATA_DIR, 'lst')
PROCESSED_VI_DATA_DIR = os.path.join(PROCESSED_DATA_DIR, 'vi')
PROCESSED_DAYMET_DATA_DIR = os.path.join(PROCESSED_DATA_DIR, 'daymet')
PROCESSED_CENSUS_DATA_DIR = os.path.join(PROCESSED_DATA_DIR, 'census')
PROCESSED_METEOSTAT_DATA_DIR = os.path.join(PROCESSED_DATA_DIR, 'meteostat')
PROCESSED_LC_DATA_DIR = os.path.join(PROCESSED_DATA_DIR, 'lc')
PROCESSED_FIS_DATA_DIR = os.path.join(PROCESSED_DATA_DIR, 'fis')

FINAL_DATA_DIR = os.path.join(PROCESSED_DATA_DIR, 'final')

CITIES = {
    "ALBUQUERQUE": {
        "state": "NM",
        "zipcodes": [
            '87101', '87102', '87103', '87104', '87105', '87106', '87107', '87108', '87109',
            '87110', '87111', '87112', '87113', '87114', '87120', '87121', '87122', '87123',
            '87131', '87151', '87153', '87154', '87158', '87174', '87176', '87181', '87184',
            '87185', '87187', '87190', '87191', '87192', '87193', '87194', '87195', '87196',
            '87197', '87198', '87199'
        ],
        "modis": ["h09v05"],
        "daymet": ["11197"],
        "latitude": 35.0844,
        "longitude": -106.6504,
    },
    "COLUMBUS": {
        "state": "OH",
        "zipcodes": [
            '43085', '43201', '43202', '43203', '43204', '43205', '43206', '43207', '43209',
            '43210', '43211', '43212', '43213', '43214', '43215', '43217', '43219', '43220',
            '43221', '43222', '43223', '43224', '43227', '43228', '43229', '43230', '43231',
            '43232', '43235', '43240'
        ],
        "modis": ["h11v05"],
        "daymet": ["11569", "11749"],
        "latitude": 39.9612,
        "longitude": -82.9988,
    },
    "DENVER": {
        "state": "CO",
        "zipcodes": [
            '80202', '80203', '80204', '80205', '80206', '80207', '80209', '80210', '80211',
            '80212', '80216', '80218', '80219', '80220', '80222', '80223', '80224', '80226',
            '80227', '80230', '80231', '80232', '80235', '80236', '80237', '80238', '80239',
            '80246', '80249', '80264', '80290', '80293', '80294', '80299'
        ],
        "modis": ["h09v05"],
        "daymet":["11558"],
        "latitude": 39.7392,
        "longitude": -104.9903,
    },
    "LAS VEGAS": {
        "state": "NV",
        "zipcodes": [
            '89101', '89102', '89103', '89104', '89106', '89107', '89108', '89109', '89110',
            '89117', '89118', '89119', '89120', '89121', '89122', '89123', '89124', '89128',
            '89129', '89130', '89131', '89134', '89135', '89138', '89139', '89141', '89142',
            '89143', '89144', '89145', '89146', '89147', '89148', '89149', '89156', '89161',
            '89166', '89169', '89178', '89179', '89183'
        ],
        "modis": ["h08v05"],
        "daymet": ["11193", "11373"],
        "latitude": 36.1699,
        "longitude": -115.1398,
    },
    "SEATTLE": {
        "state": "WA",
        "zipcodes": [
            '98101', '98102', '98103', '98104', '98105', '98106', '98107', '98108', '98109',
            '98112', '98115', '98116', '98117', '98118', '98119', '98121', '98122', '98125',
            '98126', '98133', '98134', '98136', '98144', '98154', '98164', '98174', '98195',
            '98199'
        ],
        "modis": ["h09v04"],
        "daymet": ["12269"],
        "latitude": 47.6062,
        "longitude": -122.3321,
    },
}

## Processing Logic

In [150]:
def get_fs_name(fs_path):
    return fs_path.split("/")[-1]
    
def generate_date_chunks(years, chunk_size):
    all_dates = []
    for year in years:
        start_date = datetime.date(year, 1, 1)
        end_date = datetime.date(year, 12, 31)
        current_date = start_date
        while current_date <= end_date:
            chunk = []
            for _ in range(chunk_size):
                if current_date > end_date:
                    break
                chunk.append(current_date)
                current_date += datetime.timedelta(days=1)
            all_dates.append(chunk)
    return all_dates

def download_zipcode_data():
    zip_url = "https://www2.census.gov/geo/tiger/TIGER2024/ZCTA520/tl_2024_us_zcta520.zip"
    output_dir = ZCTA_DATA_DIR
    os.makedirs(output_dir, exist_ok=True)
    print(f"Downloading {zip_url}...")
    response = requests.get(zip_url, stream=True)
    response.raise_for_status()
    print("Extracting files...")
    with zipfile.ZipFile(BytesIO(response.content)) as z:
        z.extractall(output_dir)

def get_granule_links_by_coordinates(lat, lon, start_date, end_date, product_name):
    api_url = "https://cmr.earthdata.nasa.gov/search/granules.json"
    links = []
    token = NASA_EARTHDATA_TOKEN
    params = {
        "short_name": product_name,
        "bounding_box": f"{lon-0.01},{lat-0.01},{lon+0.01},{lat+0.01}",
        "temporal": f"{start_date}T00:00:00Z,{end_date}T23:59:59Z",
        "page_size": 20,
        "token": token
    }
    response = requests.get(api_url, params=params)
    response.raise_for_status()
    granules = response.json().get("feed", {}).get("entry", [])
    print(f"Found {len(granules)} granules.")
    for granule in granules:
        for link in granule["links"]: 
            link_title = link.get("title")
            if link_title is not None and "Download " in link_title and "hdf" in link_title:
                links.append(link["href"])
    return links

def download_granule(granule_url, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    token = NASA_EARTHDATA_TOKEN
    granule_filename = granule_url.split("/")[-1]
    headers = {"Authorization": f"Bearer {token}"}
    with requests.get(granule_url, headers=headers, stream=True) as response:
        response.raise_for_status()
        granule_path = os.path.join(output_dir, granule_filename)
        with open(granule_path, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
    
def download_modis_data(product_name, output_dir):
    years = [2022, 2023]
    chunk_size = 20
    chunks = generate_date_chunks(years, chunk_size)
    for chunk in chunks:
        chunk_str = [datetime.datetime.strftime(x, "%Y-%m-%d") for x in chunk]
        start, end = chunk_str[0], chunk_str[-1]
        for city, conf in CITIES.items():
            granule_links = get_granule_links_by_coordinates(conf["latitude"], conf["longitude"], start, end, product_name=product_name)
            for link in tqdm(granule_links, desc=f"Downloading MOD11A1 files for {city} [{start}, {end}]"):
                download_granule(link, output_dir)
                
def download_lst_data():
    download_modis_data("MOD11A1", LST_DATA_DIR)

def download_vi_data():
    download_modis_data("MOD13A2", VI_DATA_DIR)
    
def get_observation_date(hdf_file):
    basename = os.path.basename(hdf_file)
    parts = basename.split('.')
    date_str = parts[1]  
    year = int(date_str[1:5])       
    doy = int(date_str[5:8])        
    measurement_date = datetime.datetime(year, 1, 1) + datetime.timedelta(doy - 1)
    return measurement_date

def get_zip_avg_lst(hdf_file, zip_codes):
    
    hdf_ds = gdal.Open(hdf_file)
    subdatasets = hdf_ds.GetSubDatasets()
    lst_day_sds_name = None
    for sds_name, sds_desc in subdatasets:
        if 'LST_Day_1km' in sds_name:
            lst_day_sds_name = sds_name
            break
    if lst_day_sds_name is None:
        raise Exception('LST_Day_1km subdataset not found.')
    lst_day_ds = gdal.Open(lst_day_sds_name)
    
    lst_day_array = lst_day_ds.ReadAsArray()
    nodata_value = lst_day_ds.GetRasterBand(1).GetNoDataValue()
    
    lst_data = np.where(lst_day_array == nodata_value, np.nan, lst_day_array)
    lst_data = lst_data * 0.02  
    lst_data = lst_data - 273.15  
    
    output_raster = 'tmp_lst_day_celsius.tif'
    driver = gdal.GetDriverByName('GTiff')
    out_ds = driver.Create(
        output_raster,
        lst_day_ds.RasterXSize,
        lst_day_ds.RasterYSize,
        1,
        gdal.GDT_Float32
    )
    out_ds.SetGeoTransform(lst_day_ds.GetGeoTransform())
    out_ds.SetProjection(lst_day_ds.GetProjection())
    outband = out_ds.GetRasterBand(1)
    outband.WriteArray(lst_data)
    outband.SetNoDataValue(np.nan)
    outband.FlushCache()
    out_ds = None
    
    reprojected_raster = 'lst_day_epsg4326_celsius.tif'
    gdal.Warp(
        reprojected_raster,
        output_raster,
        dstSRS='EPSG:4326',
        format='GTiff',
        resampleAlg='bilinear',
        dstNodata=np.nan
    )

    zcta_file = ZCTA_SHAPEFILE
    zcta_gdf = gpd.read_file(zcta_file)
    target_zctas = zcta_gdf[zcta_gdf['ZCTA5CE20'].isin(zip_codes)]
    target_zctas = target_zctas.to_crs('EPSG:4326')

    stats = zonal_stats(
        target_zctas,
        reprojected_raster,
        stats=['mean'],
        nodata=np.nan,
        geojson_out=True
    )
    
    zonal_gdf = gpd.GeoDataFrame.from_features(stats)
    target_zctas = target_zctas.merge(
        zonal_gdf[['ZCTA5CE20', 'mean']],
        on='ZCTA5CE20',
        how='left'
    )
    
    target_zctas.rename(columns={'mean': 'avg_LST_C'}, inplace=True)
    return target_zctas

def concatenate_geojson_files(city_name, directory, output_directory):

    file_pattern = os.path.join(directory, f'{city_name}_*.geojson')
    files = glob.glob(file_pattern)
    
    target_columns = ['ZCTA5CE20', 'avg_LST_C', 'geometry']
    
    if not files:
        print(f"No files found for {city_name} in {get_fs_name(directory)}.")
        return
    
    gdfs = []
    
    for file in tqdm(files, desc=f"Processing {city_name} files"):
        filename = os.path.basename(file)
        date_str = filename.replace(f"{city_name}_","").replace(".geojson", "")
        
        try:
            observation_date = datetime.datetime.strptime(date_str, '%Y%m%d')
        except ValueError:
            print(f"Error parsing date from filename {get_fs_name(filename)}. Expected format is {city_name}_YYYYMMDD.geojson")
            continue

        try:
            gdf = gpd.read_file(file)
        except Exception as e:
            print(f"Error reading {get_fs_name(filename)}: {e}")
            continue
        
        available_columns = [col for col in target_columns if col in gdf.columns]
        if not available_columns:
            print(f"No target columns found in {get_fs_name(filename)}. Skipping.")
            continue
        gdf = gdf[available_columns].copy()
        
        if 'avg_LST_C' not in gdf.columns:
            print(f"'avg_LST_C' column not found in {get_fs_name(filename)}. Skipping.")
            continue
        
        gdf['avg_LST_C'] = gdf['avg_LST_C'].apply(lambda x: np.nan if x is None else x)        
        gdf['avg_LST_C'] = pd.to_numeric(gdf['avg_LST_C'], errors='coerce')
        gdf.fillna(np.nan, inplace=True)
        gdf['observation_date'] = observation_date
        gdfs.append(gdf)
    
    if not gdfs:
        print(f"No valid GeoDataFrames to concatenate for {city_name}.")
        return
    
    combined_gdf = pd.concat(gdfs, ignore_index=True)
    combined_gdf = gpd.GeoDataFrame(combined_gdf, geometry='geometry', crs='EPSG:4326')
    output_file = os.path.join(output_directory, f'{city_name}.geojson')
    try:
        combined_gdf.to_file(output_file, driver='GeoJSON')
        print(f"Combined GeoJSON saved as {get_fs_name(output_file)}.")
    except Exception as e:
        print(f"Error saving combined GeoJSON for {city_name}: {e}")
        
        
def format_city_name(filename):
    return os.path.splitext(os.path.basename(filename))[0].replace("_", " ").upper()


def consolidate_geojson(input_dir, output_file):
    file_names = ["albuquerque.geojson", "seattle.geojson", "las_vegas.geojson", "columbus.geojson", "denver.geojson"]
    files = [os.path.join(input_dir, file_name) for file_name in file_names]

    if not files:
        print(f"No GeoJSON files found in directory: {get_fs_name(input_dir)}")
        return

    gdf_list = []
    for file in files:
        try:
            print(f"Processing file: {get_fs_name(file)}")
            gdf = gpd.read_file(file)
            city_name = format_city_name(file)
            gdf['city'] = city_name
            gdf_list.append(gdf)
        except Exception as e:
            print(f"Error processing {get_fs_name(file)}: {e}")

    if not gdf_list:
        print("No GeoJSON data to consolidate.")
        return

    consolidated_gdf = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True), crs=gdf_list[0].crs)
    consolidated_gdf.to_file(output_file, driver='GeoJSON')
    print(f"Consolidated GeoJSON saved to: {get_fs_name(output_file)}")

def get_zip_avg_vi(hdf_file, zip_codes, vi_type='NDVI'):
    hdf_ds = gdal.Open(hdf_file)
    subdatasets = hdf_ds.GetSubDatasets()
    vi_sds_name = None

    for sds_name, sds_desc in subdatasets:
        if vi_type in sds_name:
            vi_sds_name = sds_name
            break

    if vi_sds_name is None:
        raise Exception(f'{vi_type} subdataset not found.')

    vi_ds = gdal.Open(vi_sds_name)

    vi_array = vi_ds.ReadAsArray()
    nodata_value = vi_ds.GetRasterBand(1).GetNoDataValue()

    vi_data = np.where(vi_array == nodata_value, np.nan, vi_array)

    if vi_type == 'NDVI' or vi_type == 'EVI':
        scale_factor = 0.0001  
        vi_data = vi_data * scale_factor  
    else:
        raise ValueError("Unsupported Vegetation Index type. Use 'NDVI' or 'EVI'.")

    if vi_type == 'NDVI':
        vi_data = np.clip(vi_data, -1, 1)
    elif vi_type == 'EVI':
        vi_data = np.clip(vi_data, -1, 1)

    output_raster = 'tmp_vi.tif'
    driver = gdal.GetDriverByName('GTiff')
    out_ds = driver.Create(
        output_raster,
        vi_ds.RasterXSize,
        vi_ds.RasterYSize,
        1,
        gdal.GDT_Float32
    )
    out_ds.SetGeoTransform(vi_ds.GetGeoTransform())
    out_ds.SetProjection(vi_ds.GetProjection())
    outband = out_ds.GetRasterBand(1)
    outband.WriteArray(vi_data)
    outband.SetNoDataValue(np.nan)
    outband.FlushCache()
    out_ds = None  

    reprojected_raster = 'vi_epsg4326.tif'
    gdal.Warp(
        reprojected_raster,
        output_raster,
        dstSRS='EPSG:4326',
        format='GTiff',
        resampleAlg='bilinear',
        dstNodata=np.nan
    )
    
    zcta_file = ZCTA_SHAPEFILE
    zcta_gdf = gpd.read_file(zcta_file)
    target_zctas = zcta_gdf[zcta_gdf['ZCTA5CE20'].isin(zip_codes)]
    target_zctas = target_zctas.to_crs('EPSG:4326')

    stats = zonal_stats(
        target_zctas,
        reprojected_raster,
        stats=['mean'],
        nodata=np.nan,
        geojson_out=True
    )

    zonal_gdf = gpd.GeoDataFrame.from_features(stats)
    
    target_zctas = target_zctas.merge(
        zonal_gdf[['ZCTA5CE20', 'mean']],
        on='ZCTA5CE20',
        how='left'
    )

    target_zctas.rename(columns={'mean': f'avg_{vi_type}'}, inplace=True)
    return target_zctas

def consolidate_geojson_files(ready_directory, output_file, vi_type='NDVI'):

    pattern = os.path.join(ready_directory, f'*_{vi_type}_*.geojson')
    files = glob.glob(pattern)

    if not files:
        print(f"No GeoJSON files found for {vi_type} in directory: {get_fs_name(ready_directory)}")
        return

    gdf_list = []
    for file in files:
        try:
            print(f"Processing file: {get_fs_name(file)}")
            gdf = gpd.read_file(file)

            basename = os.path.basename(file)
            parts = basename.rsplit('_', 2)  
            
            if len(parts) != 3:
                raise ValueError(f"Filename {basename} does not match the expected pattern.")
            
            city = parts[0].upper()  
            date_part = parts[2].split('.')[0]  
            
            date = datetime.datetime.strptime(date_part, "%Y%m%d").date()
            
            gdf['CITY'] = city
            gdf['DATE'] = date
            
            gdf_list.append(gdf)
        except Exception as e:
            print(f"Error processing {get_fs_name(file)}: {e}")

    if not gdf_list:
        print("No GeoJSON data to consolidate.")
        return

    consolidated_gdf = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True), crs=gdf_list[0].crs)

    # output_file = os.path.join(ready_directory, f'cities_consolidated_{vi_type}.geojson')
    consolidated_gdf.to_file(output_file, driver='GeoJSON')
    print(f"Consolidated GeoJSON saved to: {get_fs_name(output_file)}")
    
def process_lst_data(source_dir, output_file):
    directory = f"{source_dir}/"
    ready_directory = f"{source_dir}/stage/"
    os.makedirs(ready_directory, exist_ok=True)
    os.makedirs(PROCESSED_LST_DATA_DIR, exist_ok=True)
    files = os.listdir(directory)
    cnt = 0

    for modis_file in tqdm(files, desc="Processing MODIS LST files"):
        modis_file_full = os.path.join(directory, modis_file)

        if os.path.isfile(modis_file_full):
            observ_date = datetime.datetime.strftime(get_observation_date(modis_file), "%Y-%m-%d")

            try:
                if "MOD11A1" in modis_file and "h08v05" in modis_file:
                    las_vegas_day_lst = get_zip_avg_lst(modis_file_full, CITIES["LAS VEGAS"]["zipcodes"])
                    las_vegas_day_lst.to_file(f'''{ready_directory}las_vegas_{observ_date.replace('-', '')}.geojson''', driver='GeoJSON')
                elif "MOD11A1" in modis_file and"h09v04" in modis_file:
                    seattle_day_lst = get_zip_avg_lst(modis_file_full, CITIES["SEATTLE"]["zipcodes"])
                    seattle_day_lst.to_file(f'''{ready_directory}seattle_{observ_date.replace('-', '')}.geojson''', driver='GeoJSON')                 
                elif "MOD11A1" in modis_file and "h09v05" in modis_file:
                    albuquerque_day_lst = get_zip_avg_lst(modis_file_full, CITIES["ALBUQUERQUE"]["zipcodes"])
                    albuquerque_day_lst.to_file(f'''{ready_directory}albuquerque_{observ_date.replace('-', '')}.geojson''', driver='GeoJSON')        
                    denver_day_lst = get_zip_avg_lst(modis_file_full, CITIES["DENVER"]["zipcodes"])
                    denver_day_lst.to_file(f'''{ready_directory}denver_{observ_date.replace('-', '')}.geojson''', driver='GeoJSON')
                elif "MOD11A1" in modis_file and "h11v05" in modis_file:
                    columbus_day_vi = get_zip_avg_lst(modis_file_full, CITIES["COLUMBUS"]["zipcodes"])
                    columbus_day_vi.to_file(f'''{ready_directory}columbus_{observ_date.replace('-', '')}.geojson''', driver='GeoJSON')
                cnt += 1
            except Exception as e:
                print(f"Error processing {get_fs_name(modis_file)}: {e}")

    concatenate_geojson_files('seattle', ready_directory, ready_directory)
    concatenate_geojson_files('las_vegas', ready_directory, ready_directory)
    concatenate_geojson_files('albuquerque', ready_directory, ready_directory)
    concatenate_geojson_files('denver', ready_directory, ready_directory)
    concatenate_geojson_files('columbus', ready_directory, ready_directory)

    geojson_file = f"{ready_directory}/{output_file.split('/')[-1].split('.')[0]}.geojson"
    consolidate_geojson(ready_directory, geojson_file)

    print(f"Processed {cnt} files.")
    output_csv_path = output_file
    print("Loading GeoJSON file...")
    gdf = gpd.read_file(geojson_file)
    print("Dropping geometry column...")
    df = gdf.drop(columns='geometry')
    print("Saving to CSV...")
    df.to_csv(output_csv_path, index=False)
    print(f"CSV file saved at '{get_fs_name(output_csv_path)}'.")
    
    input_csv_path = output_csv_path
    output_csv_path = output_csv_path.replace("cities.csv", "cities_new.csv")
    print("Loading CSV file...")
    df = pd.read_csv(input_csv_path)
    print("Renaming column 'ZCTA5CE20' to 'zipcode'...")
    df.rename(columns={"ZCTA5CE20": "zipcode"}, inplace=True)
    df_cleaned = df.dropna(subset=["avg_LST_C"])
    df_cleaned = df_cleaned.rename(columns={'avg_LST_C': 'avg_lst_c', 'observation_date': 'date'})
    print("Saving updated CSV...")
    df_cleaned.to_csv(output_csv_path, index=False)
    print(f"Updated CSV saved at '{get_fs_name(output_csv_path)}'.")
    
    
def process_vi_data(source_dir, output_file):
    directory = f"{source_dir}/"
    ready_directory = f"{source_dir}/stage/"
    os.makedirs(ready_directory, exist_ok=True)
    os.makedirs(PROCESSED_VI_DATA_DIR, exist_ok=True)
    files = os.listdir(directory)

    cnt = 0

    for modis_file in tqdm(files, desc="Processing MODIS VI files"):
        modis_file_full = os.path.join(directory, modis_file)
        if os.path.isfile(modis_file_full):
            observ_date = datetime.datetime.strftime(get_observation_date(modis_file), "%Y-%m-%d")

            try:

                if "MOD13A2" in modis_file and "h08v05" in modis_file:
                    las_vegas_day_vi = get_zip_avg_vi(modis_file_full, CITIES["LAS VEGAS"]["zipcodes"], vi_type='NDVI')
                    las_vegas_day_vi.to_file(f'''{ready_directory}las_vegas_NDVI_{observ_date.replace('-', '')}.geojson''', driver='GeoJSON')
                elif "MOD13A2" in modis_file and "h09v04" in modis_file:
                    seattle_day_vi = get_zip_avg_vi(modis_file_full, CITIES["SEATTLE"]["zipcodes"], vi_type='NDVI')
                    seattle_day_vi.to_file(f'''{ready_directory}seattle_NDVI_{observ_date.replace('-', '')}.geojson''', driver='GeoJSON')                 
                elif "MOD13A2" in modis_file and "h09v05" in modis_file:
                    albuquerque_day_vi = get_zip_avg_vi(modis_file_full, CITIES["ALBUQUERQUE"]["zipcodes"], vi_type='NDVI')
                    albuquerque_day_vi.to_file(f'''{ready_directory}albuquerque_NDVI_{observ_date.replace('-', '')}.geojson''', driver='GeoJSON')        
                    denver_day_vi = get_zip_avg_vi(modis_file_full, CITIES["DENVER"]["zipcodes"], vi_type='NDVI')
                    denver_day_vi.to_file(f'''{ready_directory}denver_NDVI_{observ_date.replace('-', '')}.geojson''', driver='GeoJSON')
                elif "MOD13A2" in modis_file and "h11v05" in modis_file:
                    columbus_day_vi = get_zip_avg_vi(modis_file_full, CITIES["COLUMBUS"]["zipcodes"], vi_type='NDVI')
                    columbus_day_vi.to_file(f'''{ready_directory}columbus_NDVI_{observ_date.replace('-', '')}.geojson''', driver='GeoJSON')

                cnt += 1
            except Exception as e:
                print(f"Error processing {get_fs_name(modis_file)}: {e}")

    print(f"Processed {cnt} files.")

    geojson_file = f"{ready_directory}/{output_file.split('/')[-1].split('.')[0]}.geojson"
    consolidate_geojson_files(ready_directory, geojson_file, vi_type='NDVI')
    
    output_csv_path = output_file
    print("Loading GeoJSON file...")
    gdf = gpd.read_file(geojson_file)
    print("Dropping geometry column...")
    df = gdf.drop(columns='geometry')
    df.rename(columns={"ZCTA5CE20": "zipcode"}, inplace=True)
    df.rename(columns={"DATE": "date"}, inplace=True)
    df.rename(columns={"CITY": "city"}, inplace=True)
    df.rename(columns={"avg_NDVI": "avg_ndvi"}, inplace=True)
    df = df[["zipcode", "date", "avg_ndvi", "city"]]
    df.columns = df.columns.str.strip()
    date_range = pd.date_range(start="2022-01-01", end="2023-12-31").strftime('%Y-%m-%d')
    unique_zipcodes = df['zipcode'].unique()
    full_combinations = pd.DataFrame(
        list(pd.MultiIndex.from_product([unique_zipcodes, date_range], names=["zipcode", "date"])),
        columns=["zipcode", "date"]
    )
    df['date'] = pd.to_datetime(df['date'], errors='coerce').dt.strftime('%Y-%m-%d')
    full_combinations['date'] = full_combinations['date'].astype(str)
    result_df = pd.merge(
        full_combinations,
        df,
        how="left",
        on=["zipcode", "date"]
    )
    result_df['avg_ndvi'] = result_df.groupby('zipcode')['avg_ndvi'].ffill()
    result_df['city'] = result_df.groupby('zipcode')['city'].ffill()
    result_df.dropna(subset=['avg_ndvi', 'city'], inplace=True)
    result_df.dropna(inplace=True)
    print("Saving to CSV...")
    result_df.to_csv(output_csv_path, index=False)
    print(f"""CSV file saved at '{get_fs_name(output_csv_path)}'.""")

def combine_daymet_data(stage_dir):
    meteo_sat_dir = PROCESSED_DAYMET_DATA_DIR
    output_csv_path = os.path.join(meteo_sat_dir, "combined_meteo_sat_data.csv")
    city_mapping = {
        "ALBUQUERQUE": "ALBUQUERQUE",
        "COLUMBUS": "COLUMBUS",
        "DENVER": "DENVER",
        "LAS_VEGAS": "LAS_VEGAS",
        "SEATTLE": "SEATTLE",
    }

    dataframes = []
    
    print("Processing files...")
    for file_name in os.listdir(stage_dir):
        if file_name.endswith(".csv"):
            for city_key, city_name in city_mapping.items():
                if city_key in file_name:
                    file_path = os.path.join(stage_dir, file_name)
                    variable = file_name.split("_avg_")[-1].replace(".csv", "")
                    df = pd.read_csv(file_path)
                    df.rename(columns={df.columns[-1]: variable}, inplace=True)
                    df["city"] = city_name
                    dataframes.append(df)
                    break
    
    print("Combining data...")
    if dataframes:
        combined_df = pd.concat(dataframes, ignore_index=True)
        combined_df = combined_df.pivot_table(
            index=["time", "zipcode", "city"], 
            columns=[], 
            values=list(combined_df.columns.difference(["time", "zipcode", "city"]))
        ).reset_index()
        print("Saving combined data to CSV...")
        combined_df['time'] = pd.to_datetime(combined_df['time']).dt.date  
        combined_df.rename(columns={'time': 'date'}, inplace=True) 
        combined_df.to_csv(output_csv_path, index=False)
        print(f"""Combined data saved to '{get_fs_name(output_csv_path)}'.""")
    else:
        print("No files to process.")
        
        
        
def process_daymet_data(source_dir):
    stage_directory = f"{source_dir}/stage/"
    os.makedirs(stage_directory, exist_ok=True)
    os.makedirs(PROCESSED_DAYMET_DATA_DIR, exist_ok=True)
    variables = ['dayl', 'prcp', 'srad', 'swe', 'tmax', 'tmin', 'vp']
    data_dir = source_dir
    zcta_file = ZCTA_SHAPEFILE
    print("Loading ZIP code shapefile...")
    zcta_gdf = gpd.read_file(zcta_file)
    for city, conf in CITIES.items():
        print(f"Processing city: {city}")
        zip_codes = conf.get("zipcodes",[])
        tiles = conf.get("daymet",[])
        if not zip_codes:
            print(f"No zip codes found for city: {city}")
            continue
        city_zcta_gdf = zcta_gdf[zcta_gdf['ZCTA5CE20'].isin(zip_codes)].copy()
        if city_zcta_gdf.empty:
            print(f"No matching ZIP codes found in shapefile for city: {city}")
            continue
        if city_zcta_gdf.crs is None:
            print(f"ZIP code shapefile for city {city} has no CRS. Skipping.")
            continue
        city_variable_dfs = {var: [] for var in variables}
        for tile in tiles:
            print(f"  Processing tile: {tile}")
            # Iterate through each variable
            for var in variables:
                # Find files for this tile and variable
                var_pattern = os.path.join(data_dir, f"{tile}_*_{var}.nc")
                var_files = glob.glob(var_pattern)
                if not var_files:
                    print(f"    No files found for variable '{var}' in tile '{tile}'.")
                    continue
                for var_file in var_files:
                    print(f"    Processing file: {get_fs_name(var_file)}")
                    try:
                        dataset = xr.open_dataset(var_file)
                        if var not in dataset:
                            print(f"      Variable '{var}' not found in file '{get_fs_name(var_file)}'. Skipping.")
                            continue
                        variable_data = dataset[var]
                        lat = dataset['lat'].values
                        lon = dataset['lon'].values
                        x_coords = dataset['x'].values
                        y_coords = dataset['y'].values
                        df = variable_data.to_dataframe().reset_index()
                        df['y_index'] = df['y'].apply(lambda y_val: np.abs(y_coords - y_val).argmin())
                        df['x_index'] = df['x'].apply(lambda x_val: np.abs(x_coords - x_val).argmin())
                        df['lat'] = df['y_index'].apply(lambda y_idx: lat[y_idx, 0])
                        df['lon'] = df['x_index'].apply(lambda x_idx: lon[0, x_idx])
                        df = df.dropna(subset=[var])
                        gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['lon'], df['lat']), crs="EPSG:4326")
                        gdf = gdf.to_crs(city_zcta_gdf.crs)
                        joined_gdf = gpd.sjoin(gdf, city_zcta_gdf, how="inner", predicate="intersects")

                        if joined_gdf.empty:
                            print(f"      No data points found within ZIP codes for file '{get_fs_name(var_file)}'.")
                            continue
                        
                        grouped = (
                            joined_gdf.groupby(['time', 'ZCTA5CE20'])[var]
                            .mean()
                            .reset_index()
                            .rename(columns={'ZCTA5CE20': 'zipcode', var: f'avg_{var}'})
                        )
                        
                        city_variable_dfs[var].append(grouped)
                        
                    except Exception as e:
                        print(f"      Error processing file '{get_fs_name(var_file)}': {e}")
        
        for var in variables:
            if not city_variable_dfs[var]:
                print(f"  No data collected for variable '{var}' in city '{city}'. Skipping.")
                continue
            print(f"  Aggregating data for variable '{var}'")
            combined_df = pd.concat(city_variable_dfs[var], ignore_index=True)
            aggregated_df = combined_df.groupby(['time', 'zipcode'])[f'avg_{var}'].mean().reset_index()
            output_filename = f"{city}_zipcode_avg_{var}.csv"
            aggregated_df.to_csv(f"{stage_directory}/{output_filename}", index=False)
            print(f"    Saved aggregated data to '{get_fs_name(output_filename)}'")
        print(f"Finished processing city: {city}")
    print("All cities processed.")
    combine_daymet_data(stage_directory)

    

    
def process_population_data():
    os.makedirs(PROCESSED_CENSUS_DATA_DIR, exist_ok=True)
    population_file = f"{CENSUS_DATA_DIR}/DECENNIALDHC2020.P1-Data.csv"
    zcta_shapefile_path = ZCTA_SHAPEFILE
    
    output_dir = PROCESSED_CENSUS_DATA_DIR
    os.makedirs(output_dir, exist_ok=True)
    output_csv_path = os.path.join(output_dir, 'zipcode_population_density_filtered.csv')
    
    target_zip_codes = [zip_code for city, conf in CITIES.items() for zip_code in conf["zipcodes"]]    
    print("Loading population data...")
    population_df = pd.read_csv(population_file)
    population_df['ZIP'] = population_df['NAME'].str.replace("ZCTA5 ", "").str.zfill(5)
    population_df['Population'] = pd.to_numeric(population_df['P1_001N'], errors='coerce')
    population_df = population_df[population_df['ZIP'].isin(target_zip_codes)]
    print(f"Filtered population data rows: {len(population_df)}")
    print("Loading ZCTA shapefile...")
    zcta_gdf = gpd.read_file(zcta_shapefile_path)
    zcta_gdf['ZIP'] = zcta_gdf['ZCTA5CE20'].astype(str).str.zfill(5)
    zcta_gdf = zcta_gdf[zcta_gdf['ZIP'].isin(target_zip_codes)]
    print(f"Filtered ZCTA rows: {len(zcta_gdf)}")
    common_zips = set(population_df['ZIP']) & set(zcta_gdf['ZIP'])
    print(f"Common ZIP codes: {len(common_zips)}")
    
    if not common_zips:
        print("No common ZIP codes found. Please check the data formats.")
        exit()

    zcta_gdf['Area_sqkm'] = zcta_gdf.geometry.to_crs(epsg=3395).area / 1e6
    print("Merging population data with ZCTA geometry...")
    merged_gdf = zcta_gdf.merge(population_df, on='ZIP', how='inner')
    print(f"Merged rows: {len(merged_gdf)}")
    if merged_gdf.empty:
        print("Merged dataset is empty. Check for ZIP code mismatches.")
        exit()
    merged_gdf['Population_Density'] = merged_gdf['Population'] / merged_gdf['Area_sqkm']
    merged_gdf.rename(columns={"ZIP": "zipcode"}, inplace=True)
    output_df = merged_gdf[['zipcode', 'Population', 'Area_sqkm', 'Population_Density']]
    output_df.to_csv(output_csv_path, index=False)
    print(f"Filtered population density data saved to '{get_fs_name(output_csv_path)}'.")
    
def process_meteostat_data():
    os.makedirs(PROCESSED_METEOSTAT_DATA_DIR, exist_ok=True)
    meteo_dir = METEOSTAT_DATA_DIR
    output_csv_path = os.path.join(PROCESSED_METEOSTAT_DATA_DIR, "combined_meteo_data_with_city.csv")

    city_mapping = {
        "albuquerque": "ALBUQUERQUE",
        "columbus": "COLUMBUS",
        "denver": "DENVER",
        "las_vegas": "LAS VEGAS",
        "los_angeles": "LOS ANGELES",
        "portland": "PORTLAND",
        "seattle": "SEATTLE",
    }

    combined_df = pd.DataFrame()
    
    print("Combining CSV files...")
    for file_name in os.listdir(meteo_dir):
        if file_name.endswith(".csv") and file_name != "combined_meteo_data.csv":
            for city_key, city_name in city_mapping.items():
                if city_key in file_name:
                    file_path = os.path.join(meteo_dir, file_name)
                    df = pd.read_csv(file_path)
                    df["city"] = city_name
                    combined_df = pd.concat([combined_df, df], ignore_index=True)
                    break
    print("Saving combined data to CSV...")
    combined_df.to_csv(output_csv_path, index=False)
    print(f"Combined data saved to '{get_fs_name(output_csv_path)}'.")
    
def process_lc_data():
    zcta_shapefile_path = ZCTA_SHAPEFILE
    land_cover_raster_path = f"{LC_DATA_DIR}/LndCov.tif"
    output_dir = PROCESSED_LC_DATA_DIR
    os.makedirs(output_dir, exist_ok=True)

    land_cover_mapping = {
        11: 'Open Water',
        21: 'Developed, Open Space',
        22: 'Developed, Low Intensity',
        23: 'Developed, Medium Intensity',
        24: 'Developed, High Intensity',
        31: 'Barren Land',
        41: 'Deciduous Forest',
        42: 'Evergreen Forest',
        43: 'Mixed Forest',
        52: 'Shrub/Scrub',
        71: 'Grassland/Herbaceous',
        81: 'Pasture/Hay',
        82: 'Cultivated Crops',
        90: 'Woody Wetlands',
        95: 'Emergent Herbaceous Wetlands'
    }
    
    if not os.path.exists(zcta_shapefile_path):
        raise FileNotFoundError(f"ZCTA shapefile not found at {zcta_shapefile_path}")
    if not os.path.exists(land_cover_raster_path):
        raise FileNotFoundError(f"Land cover raster not found at {land_cover_raster_path}")
        
    print("Loading ZIP code shapefile...")
    zcta_gdf = gpd.read_file(zcta_shapefile_path)
    zcta_gdf['ZCTA5CE20'] = zcta_gdf['ZCTA5CE20'].astype(str).str.zfill(5)
    
    print("Original ZCTA CRS:", zcta_gdf.crs)

    if zcta_gdf.crs is None:
        zcta_gdf.set_crs("EPSG:4326", inplace=True)

    with rasterio.open(land_cover_raster_path) as src:
        raster_crs = src.crs
        print("Raster CRS:", raster_crs)
    
    if raster_crs != zcta_gdf.crs:
        print("CRS mismatch detected. Reprojecting shapefile to raster CRS...")
        zcta_gdf = zcta_gdf.to_crs(raster_crs)
        print("Reprojection complete.")
    
    print("Filtering ZCTA GeoDataFrame for target ZIP codes...")
    target_zip_codes = [zip_code for city, conf in CITIES.items() for zip_code in conf["zipcodes"]]
    zcta_filtered_gdf = zcta_gdf[zcta_gdf['ZCTA5CE20'].isin(target_zip_codes)]
    print(f"Filtered ZIP codes count: {len(zcta_filtered_gdf)}")
 
    all_land_cover_classes = [land_cover_mapping.get(code, f"Class_{code}") for code in land_cover_mapping.keys()]
    
    results = []
    print("Calculating land cover statistics...")
    for _, row in zcta_filtered_gdf.iterrows():
        zipcode = row['ZCTA5CE20']
        geometry = row['geometry']
    
        try:
            stats = zonal_stats(
                geometry,
                land_cover_raster_path,
                stats=None,
                categorical=True,
                nodata=250,
                all_touched=False
            )
            if not stats:
                continue
            class_counts = stats[0]
            class_counts.pop(250, None)  
            total_pixels = sum(class_counts.values())
            class_percentages = {
                land_cover_mapping.get(class_code, f"Class_{class_code}"): round((count / total_pixels) * 100, 2)
                for class_code, count in class_counts.items()
            }
            for land_cover_class in all_land_cover_classes:
                if land_cover_class not in class_percentages:
                    class_percentages[land_cover_class] = 0.0
            class_percentages['zipcode'] = zipcode
            results.append(class_percentages)
    
        except Exception as e:
            print(f"Error processing ZIP code {zipcode}: {e}")
    output_csv_path = os.path.join(output_dir, 'zipcode_land_cover_percentages.csv')
    land_cover_df = pd.DataFrame(results)
    cols = ['zipcode'] + all_land_cover_classes
    land_cover_df = land_cover_df.reindex(columns=cols, fill_value=0.0)
    
    land_cover_df.to_csv(output_csv_path, index=False)
    print(f"Results saved to '{get_fs_name(output_csv_path)}'.")
    
def process_fis_data():
    def calculate_isa(zcta, raster_path, pixel_area_m2):
        try:
            geometry = [zcta['geometry'].__geo_interface__]
            stats = zonal_stats(
                geometry,
                raster_path,
                stats=["sum", "count"],
                nodata=250.0,  
                all_touched=False,  
                raster_out=False
            )
            
            if not stats:
                return {'zipcode': zcta['ZCTA5CE20'], 'total_impervious_km2': 0, 'percent_impervious': 0}
            
            impervious_sum = stats[0]['sum']  
            count = stats[0]['count']  
            if count == 0:
                return {'zipcode': zcta['ZCTA5CE20'], 'total_impervious_km2': 0, 'percent_impervious': 0}
            mean_impervious = impervious_sum / count  # In percentage
            
            if mean_impervious > 100:
                mean_impervious = (mean_impervious / 255) * 100  # Convert to percentage
            total_impervious_km2 = (mean_impervious / 100) * (pixel_area_m2 * count) / 1e6
            
            percent_impervious = mean_impervious
            
            percent_impervious = min(percent_impervious, 100.0)
            
            return {
                'zipcode': zcta['ZCTA5CE20'],
                'total_impervious_km2': round(total_impervious_km2, 4),
                'percent_impervious': round(percent_impervious, 2)
            }
            
        except Exception as e:
            print(f"Error processing ZIP code {zcta['ZCTA5CE20']}: {e}")
            return {'zipcode': zcta['ZCTA5CE20'], 'total_impervious_km2': np.nan, 'percent_impervious': np.nan}


    nlcd_raster_path = f'{FIS_DATA_DIR}/FctImp.tif'
    zcta_shapefile_path = ZCTA_SHAPEFILE
    
    output_dir = PROCESSED_FIS_DATA_DIR
    os.makedirs(output_dir, exist_ok=True)
    
    print("Loading ZIP code shapefile...")
    zcta_gdf = gpd.read_file(zcta_shapefile_path)
    
    zcta_gdf['ZCTA5CE20'] = zcta_gdf['ZCTA5CE20'].astype(str).str.zfill(5)
    
    with rasterio.open(nlcd_raster_path) as src:
        nlcd_crs = src.crs
        pixel_width, pixel_height = src.res 
        raster_transform = src.transform
    
    zcta_gdf = zcta_gdf.to_crs(nlcd_crs)
    
    target_zip_codes = [zip_code for city, conf in CITIES.items() for zip_code in conf["zipcodes"]]
    zcta_filtered_gdf = zcta_gdf[zcta_gdf['ZCTA5CE20'].isin(target_zip_codes)].copy()
    
    if zcta_filtered_gdf.empty:
        print("No matching ZIP codes found in the shapefile for the specified cities.")
        exit(1)
    else:
        print(f"Filtered to {len(zcta_filtered_gdf)} ZIP codes for the specified cities.")
    
    pixel_area_m2 = pixel_width * pixel_height
    isa_metrics = []
    
    print("Calculating ISA metrics for each ZIP code...")
    for idx, zcta in zcta_filtered_gdf.iterrows():
        metrics = calculate_isa(zcta, nlcd_raster_path, pixel_area_m2)
        isa_metrics.append(metrics)

    isa_df = pd.DataFrame(isa_metrics)

    output_csv_path = os.path.join(output_dir, 'zipcode_impervious_surface_area_corrected.csv')
    isa_df.to_csv(output_csv_path, index=False)
    
    print(f"Corrected ISA metrics saved to '{get_fs_name(output_csv_path)}'")
    
def create_final_dataset():
    os.makedirs(FINAL_DATA_DIR, exist_ok=True)
    lst_file = os.path.join(PROCESSED_LST_DATA_DIR, "cities_lst.csv")
    meteo_per_city_file = os.path.join(PROCESSED_METEOSTAT_DATA_DIR, "combined_meteo_data_with_city.csv")
    meteo_sat_file = os.path.join(PROCESSED_DAYMET_DATA_DIR, "combined_meteo_sat_data.csv")
    population_file = os.path.join(PROCESSED_CENSUS_DATA_DIR, "zipcode_population_density_filtered.csv")
    vegetation_file = os.path.join(PROCESSED_VI_DATA_DIR, "cities_vi.csv")
    impervious_surface_file = os.path.join(PROCESSED_FIS_DATA_DIR, "zipcode_impervious_surface_area_corrected.csv")
    land_cover_file = os.path.join(PROCESSED_LC_DATA_DIR, "zipcode_land_cover_percentages.csv")
    output_csv_path = "data/processed/uhi.csv"
    print("Loading files...")
    lst_df = pd.read_csv(lst_file)
    meteo_sat_df = pd.read_csv(meteo_sat_file)
    vegetation_df = pd.read_csv(vegetation_file)
    meteo_per_city_df = pd.read_csv(meteo_per_city_file)
    population_df = pd.read_csv(population_file)
    impervious_surface_df = pd.read_csv(impervious_surface_file)
    land_cover_df = pd.read_csv(land_cover_file)
    print("Selecting required columns from meteo_per_city...")
    meteo_per_city_df = meteo_per_city_df[['date', 'city', 'wdir', 'wspd', 'pres']]
    print(f"Merging lst_df with meteo_sat_df ...")
    df_merged = lst_df.merge(meteo_sat_df, on=["zipcode", "date"], how="inner")
    df_merged = df_merged.merge(vegetation_df, on=["zipcode", "date"], how="inner")
    df_merged = df_merged.merge(meteo_per_city_df, on=["city", "date"], how="inner")
    population_df.rename(columns={"ZIP": "zipcode"}, inplace=True)
    df_merged = df_merged.merge(population_df[["zipcode", "Population_Density"]], on=["zipcode"], how="inner")
    df_merged = df_merged.merge(impervious_surface_df[["zipcode", "percent_impervious"]], on=["zipcode"], how="inner")
    df_merged = df_merged.merge(land_cover_df, on=["zipcode"], how="inner")
    df_merged['min_lst'] = df_merged.groupby(['date', 'city'])['avg_lst_c'].transform('min')
    df_merged['lst_diff'] = df_merged['avg_lst_c'] - df_merged['min_lst']
    df_merged['uhi_observed'] = df_merged['lst_diff'].apply(lambda x: 'Y' if x > 5 else 'N')
    df_merged.drop(['min_lst', 'lst_diff', 'city_x', 'city_y', 'srad', 'swe'], axis=1, inplace=True)
    df_merged.rename(columns={
        "dayl": "Day_length_sec",
        "prcp": "precipitation_mm",
        "tmax": "Max_air_temp_c",
        "tmin": "Min_air_temp_c",
        "vp": "Water_vapor_pressure_pa",
        "wdir": "wind_dir_degrees",
        "wspd": "wind_speed_kmh",
        "pres": "atm_pressure_hpa",
        "Open Water": "open_water_pct",
        "Developed, Open Space": "developed_open_space_pct",
        "Developed, Low Intensity": "Developed_Low_Intensity_pct",
        "Developed, Medium Intensity": "Developed_Medium_Intensity_pct",
        "Developed, High Intensity": "Developed_High_Intensity_pct",
        "Barren Land": "Barren_Land_pct",
        "Deciduous Forest": "Deciduous_Forest_pct",
        "Evergreen Forest": "Evergreen_Forest_pct",
        "Mixed Forest": "Mixed_Forest_pct",
        "Shrub/Scrub": "Shrub_Scrub_pct",
        "Grassland/Herbaceous": "Grassland_Herbaceous_pct",
        "Pasture/Hay": "Pasture_Hay_pct",
        "Cultivated Crops": "Cultivated_Crops",
        "Woody Wetlands": "Woody_Wetlands_pct",
        "Emergent Herbaceous Wetlands": "Emergent_Herbaceous_Wetlands_pct"
    }, inplace=True)
    df_merged.to_csv(output_csv_path, index=False)
    print(f"""Dataset saved to '{get_fs_name(output_csv_path)}'""")



## Downloading and Processing Data

### Step 1. Download a geographical shapefile with ZIP code service areas boundaries

The TIGER/Line Shapefiles are a core geographic product from the Census Bureau that contain extracts of geographic and cartographic information from the MAF/TIGER database. They  are approximate representations of ZIP Code service areas created by the Census Bureau to present statistical data. 



In [137]:
download_zipcode_data()

Extracting files...


### Step 2. Download MODIS Land Surface Temperature data

The MODIS MOD11A1 product is a daily, 1km resolution dataset from the Terra satellite's Moderate Resolution Imaging Spectroradiometer (MODIS) that provides Land Surface Temperature (LST) and Emissivity data for each pixel on Earth, essentially giving a detailed picture of the surface temperature across the globe with high spatial accuracy; it includes quality control flags to indicate data reliability and is commonly used in studies related to climate, land use, and environmental monitoring.

- Each pixel in the grid represents the land surface temperature at that location. 
- Quality control flags indicate:
  - Whether the pixel data is reliable (clear sky).
  - Whether the pixel data is potentially affected by clouds.
- Multiple observations** may be averaged for a single pixel, especially at higher latitudes where the satellite may pass over the same area multiple times.

Format: HDFv4

To cover the cities used in our modeling we need to download 16GB of data.
It takes long time to complete the download, so I use earlier downloaded date and do not repeat it here.



In [ ]:
download_lst_data()

### Step 3. Download MODIS Normalized Difference Vegetation Index data

MODIS MOD13A2 is a NASA data product derived from the Moderate Resolution Imaging Spectroradiometer (MODIS) that provides a 16-day global composite of vegetation indices, primarily the Normalized Difference Vegetation Index (NDVI) at a 1km spatial resolution, allowing researchers to monitor vegetation health and changes over time across the globe; it uses the best available pixel value from multiple acquisitions within the 16-day period based on factors like cloud cover and view angle.

The Normalized Difference Vegetation Index (NDVI) calculated from the red and near-infrared bands of the MODIS sensor, allowing for better vegetation monitoring in areas with high biomass.

Format: HDFv4

To cover the cities used in our modeling we need to download more than 3GB of data.
It takes long time to complete the download, so I use earlier downloaded date and do not repeat it here.



In [ ]:
download_vi_data()

### Step 4. Download satellite meteo data

DAYMET is a research product that provides gridded estimates of daily weather parameters.
It uses algorithms and software to interpolate and extrapolate from daily meteorological observations to produce gridded estimates of daily weather parameters.

DAYMET data products are used in many Earth science, natural resource, biodiversity, and agricultural research areas.

Format: netCDF/Network Common Data

To cover the cities used in our modeling we need to download 2GB of data.
It takes long time to complete the download, so I use earlier downloaded date and do not repeat it here.



In [ ]:
# prepare fo daymet data download
os.makedirs(DAYMET_DATA_DIR, exist_ok=True)
os.environ["DAYMET_DATA_DIR"] = DAYMET_DATA_DIR

In [ ]:
%%sh
# download daymet data

years=(2022 2023)
variables=("dayl" "prcp" "srad" "swe" "tmax" "tmin" "vp")
tiles=(11193 11197 11373 11558 11569 11749 12269)

# Iterate through both lists
for year in "${years[@]}"; do
  for variable in "${variables[@]}"; do
    for tile in "${tiles[@]}"; do
      wget -q --show-progress --progress=bar:force --limit-rate=3m https://thredds.daac.ornl.gov/thredds/fileServer/ornldaac/2129/tiles/${year}/${tile}_${year}/${variable}.nc -O $DAYMET_DATA_DIR/${tile}_${year}_${variable}.nc
    done
  done
done



### Step 5. Download satellite Land Cover data

MRLC Land Cover product depicts the predominant thematic land cover class within the mapping year with respect to broad categories of artificial or natural surface cover.

Format: GTiff/GeoTIFF


In [138]:
# prepare for Land Cover data download
os.makedirs(LC_DATA_DIR, exist_ok=True)
os.environ["LC_DATA_DIR"] = LC_DATA_DIR

In [175]:
%%sh
echo Start download
wget -q  https://s3-us-west-2.amazonaws.com/mrlc/Annual_NLCD_LndCov_2023_CU_C1V0.tif -O $LC_DATA_DIR/LndCov.tif
echo Download complete

Start download
Download complete


### Step 6. Download satellite Fractional Impervious Surface data

The Annual National Land Cover Database (NLCD) impervious surface product provides the fractional or percent surface area of a 30-meter map pixel that is covered with processed materials or structures (pavement, concrete, rooftops, and other constructed materials) that generate surface runoff.

Format: GTiff/GeoTIFF

In [ ]:
# prepare for Fractional Impervious Surface data download
os.makedirs(FIS_DATA_DIR, exist_ok=True)
os.environ["FIS_DATA_DIR"] = FIS_DATA_DIR

In [176]:
%%sh
echo Start download
wget -q https://s3-us-west-2.amazonaws.com/mrlc/Annual_NLCD_FctImp_2023_CU_C1V0.tif -O $FIS_DATA_DIR/FctImp.tif
echo Download complete

Start download
Download complete


### Step 7. Download population density data

US Census Bureau zip code total population 2020

Format: CSV

In [141]:
%%sh
echo Start download

wget --content-disposition "https://data.census.gov/api/access/table/download?download_id=9a9fc9754aa170e9be08c6167bb61b1aa4233c29b3689db6a6f9b71ac8cb6be9" -O datasets/raw/census/DECENNIALDHC2020.P1-Data.csv.zip

unzip -j "datasets/raw/census/DECENNIALDHC2020.P1-Data.csv.zip" "DECENNIALDHC2020.P1-Data.csv" -d "datasets/raw/census"

rm datasets/raw/census/DECENNIALDHC2020.P1-Data.csv.zip

Start download


--2024-11-28 14:01:28--  https://data.census.gov/api/access/table/download?download_id=9a9fc9754aa170e9be08c6167bb61b1aa4233c29b3689db6a6f9b71ac8cb6be9
Resolving data.census.gov (data.census.gov)... 2.19.140.252
Connecting to data.census.gov (data.census.gov)|2.19.140.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 251463 (246K) [application/zip]
Saving to: ‘datasets/raw/census/DECENNIALDHC2020.P1-Data.csv.zip’

     0K .......... .......... .......... .......... .......... 20% 2.16M 0s
    50K .......... .......... .......... .......... .......... 40% 2.89M 0s
   100K .......... .......... .......... .......... .......... 61% 3.14M 0s
   150K .......... .......... .......... .......... .......... 81% 4.31M 0s
   200K .......... .......... .......... .......... .....     100% 11.2M=0.07s

2024-11-28 14:01:28 (3.41 MB/s) - ‘datasets/raw/census/DECENNIALDHC2020.P1-Data.csv.zip’ saved [251463/251463]



Archive:  datasets/raw/census/DECENNIALDHC2020.P1-Data.csv.zip
  inflating: datasets/raw/census/DECENNIALDHC2020.P1-Data.csv  


### Step 8. Daily local weather data by city

Manually download data to 2022 and 2023 for Seattle, Colorado, Denver, Albuquerque, Las Vegas and save as

- datasets/raw/meteostat/albuquerque_2022.csv
- datasets/raw/meteostat/albuquerque_2023.csv
- datasets/raw/meteostat/columbus_2022.csv
- datasets/raw/meteostat/columbus_2023.csv
- datasets/raw/meteostat/denver_2022.csv
- datasets/raw/meteostat/denver_2023.csv
- datasets/raw/meteostat/las_vegas_2022.csv
- datasets/raw/meteostat/las_vegas_2023.csv
- datasets/raw/meteostat/seattle_2022.csv
- datasets/raw/meteostat/seattle_2023.csv


### Step 9. Process Land Surface Temperature data

Original data are stored as a raster geo data in HDFv4 format.
The step extracts average LST for each ZIP code of the cities and saves the data as CSV file

In [125]:
# process LST data
process_lst_data(LST_DATA_DIR, f"{PROCESSED_LST_DATA_DIR}/cities_lst.csv")



Processing seattle files: 100%|██████████| 712/712 [00:05<00:00, 135.08it/s]


Combined GeoJSON saved as seattle.geojson.


Processing las_vegas files: 100%|██████████| 715/715 [00:16<00:00, 44.38it/s]


Combined GeoJSON saved as las_vegas.geojson.


Processing albuquerque files: 100%|██████████| 716/716 [00:06<00:00, 107.79it/s]


Combined GeoJSON saved as albuquerque.geojson.


Processing denver files: 100%|██████████| 716/716 [00:08<00:00, 83.76it/s]


Combined GeoJSON saved as denver.geojson.


Processing columbus files: 100%|██████████| 714/714 [00:10<00:00, 71.34it/s]


Combined GeoJSON saved as columbus.geojson.
Processing file: albuquerque.geojson
Processing file: seattle.geojson
Processing file: las_vegas.geojson
Processing file: columbus.geojson
Processing file: denver.geojson
Consolidated GeoJSON saved to: cities_lst.geojson
Processed 2860 files.
Loading GeoJSON file...
Dropping geometry column...
Saving to CSV...
CSV file saved at 'cities_lst.csv'.
Loading CSV file...
Renaming column 'ZCTA5CE20' to 'zipcode'...
Saving updated CSV...
Updated CSV saved at 'cities_lst.csv'.


### Step 10. Process Normalized Difference Vegetation Index data
Original data are stored as a raster geo data in HDFv4 format. The step extracts average NDVI for each ZIP code of the cities and saves the data as CSV file

In [126]:
# process VI data
process_vi_data(VI_DATA_DIR, f"{PROCESSED_VI_DATA_DIR}/cities_vi.csv")

Processing MODIS VI files: 100%|██████████| 189/189 [05:43<00:00,  1.82s/it]


Processed 188 files.
Processing file: seattle_NDVI_20230306.geojson
Processing file: seattle_NDVI_20220117.geojson
Processing file: albuquerque_NDVI_20220322.geojson
Processing file: denver_NDVI_20220525.geojson
Processing file: albuquerque_NDVI_20230813.geojson
Processing file: denver_NDVI_20220218.geojson
Processing file: columbus_NDVI_20230101.geojson
Processing file: columbus_NDVI_20230914.geojson
Processing file: seattle_NDVI_20221101.geojson
Processing file: albuquerque_NDVI_20230218.geojson
Processing file: denver_NDVI_20220813.geojson
Processing file: albuquerque_NDVI_20230525.geojson
Processing file: las_vegas_NDVI_20220610.geojson
Processing file: denver_NDVI_20230322.geojson
Processing file: columbus_NDVI_20231203.geojson
Processing file: columbus_NDVI_20231117.geojson
Processing file: denver_NDVI_20230930.geojson
Processing file: columbus_NDVI_20230306.geojson
Processing file: columbus_NDVI_20220117.geojson
Processing file: las_vegas_NDVI_20220626.geojson
Processing file: a

### Step 11. Process satellite meteo data
The data are in netCDF format. The step extracts the data and saves as CSV file

In [127]:
# process datamet data
process_daymet_data(DAYMET_DATA_DIR)

Loading ZIP code shapefile...
Processing city: ALBUQUERQUE
  Processing tile: 11197
    Processing file: 11197_2023_dayl.nc
    Processing file: 11197_2022_dayl.nc
    Processing file: 11197_2023_prcp.nc
    Processing file: 11197_2022_prcp.nc
    Processing file: 11197_2023_srad.nc
    Processing file: 11197_2022_srad.nc
    Processing file: 11197_2023_swe.nc
    Processing file: 11197_2022_swe.nc
    Processing file: 11197_2022_tmax.nc
    Processing file: 11197_2023_tmax.nc
    Processing file: 11197_2022_tmin.nc
    Processing file: 11197_2023_tmin.nc
    Processing file: 11197_2022_vp.nc
    Processing file: 11197_2023_vp.nc
  Aggregating data for variable 'dayl'
    Saved aggregated data to 'ALBUQUERQUE_zipcode_avg_dayl.csv'
  Aggregating data for variable 'prcp'
    Saved aggregated data to 'ALBUQUERQUE_zipcode_avg_prcp.csv'
  Aggregating data for variable 'srad'
    Saved aggregated data to 'ALBUQUERQUE_zipcode_avg_srad.csv'
  Aggregating data for variable 'swe'
    Saved aggre

### Step 12. Process population density data

In [128]:
process_population_data()

Loading population data...
Filtered population data rows: 150
Loading ZCTA shapefile...
Filtered ZCTA rows: 150
Common ZIP codes: 150
Merging population data with ZCTA geometry...
Merged rows: 150
Filtered population density data saved to 'zipcode_population_density_filtered.csv'.


### Step 13. Process local meteostat data

In [129]:
process_meteostat_data()

Combining CSV files...
Saving combined data to CSV...
Combined data saved to 'combined_meteo_data_with_city.csv'.


### Step 14. process Land Cover data
The data are in GeoTIFF format. The step extracts the data and saves as CSV file

In [130]:
process_lc_data()

Loading ZIP code shapefile...
Original ZCTA CRS: EPSG:4269
Raster CRS: PROJCS["AEA        WGS84",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
CRS mismatch detected. Reprojecting shapefile to raster CRS...
Reprojection complete.
Filtering ZCTA GeoDataFrame for target ZIP codes...
Filtered ZIP codes count: 150
Calculating land cover statistics...
Results saved to 'zipcode_land_cover_percentages.csv'.


### Step 15. Process Normalized Difference Vegetation Index data
The data are in GeoTIFF format. The step extracts the data and saves as CSV file

In [131]:
process_fis_data()

Loading ZIP code shapefile...
Filtered to 150 ZIP codes for the specified cities.
Calculating ISA metrics for each ZIP code...
Corrected ISA metrics saved to 'zipcode_impervious_surface_area_corrected.csv'


### Step 16. Assemble final dataset
The step combines pre-processed source data into a single dataset and saves as CSV file data/processed/uhi.csv 

In [151]:
create_final_dataset()

Loading files...
Selecting required columns from meteo_per_city...
Merging lst_df with meteo_sat_df ...
Dataset saved to 'uhi.csv'
